In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import NMT
from NMT.utils import get_dataloader

#https://tanmay17061.medium.com/load-pre-trained-glove-embeddings-in-torch-nn-embedding-layer-in-under-2-minutes-f5af8f57416a
# 
embeddings_dict = {}
with open(f"glove.6B.{dim}d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector
global_vectors = GloVe(name='6B', dim=embed_dim)
glove_weights = torch.load(f".vector_cache/glove.6B.{embed_dim}d.txt.pt")
emb_layer = nn.Embedding.from_pretrained(glove_weights[2], freeze=True, padding_idx=NMT.Constants.PAD)

In [ ]:
#Parameters
dim = 100
lstm_layers = 2
dropout = 0.2
batch_size = 32

#training params
epochs = 20
learning_rate = 1
optimizer = "SGD"
#Next options are outdated in favour of optimizer schedulers?
#max_grad_norm
#learning_rate_decay
#start_decay_at

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
# https://huggingface.co/dbmdz/bert-base-german-cased

In [ ]:
tokenizer_en = AutoTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer_de = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")

In [ ]:
from NMT import Models

encoder = Models.Encoder(num_layers=lstm_layers, bidirectional=True, dropout=dropout, rnn_size=dim)
decoder = Models.Decoder(num_layers=lstm_layers, bidirectional=False, dropout=dropout, rnn_size=dim)
model = Models.NMTModel(encoder, decoder)

In [1]:
from NMT.utils import run_evaluation, train


hidden_size = 50
batch_size = 32
num_epochs = 5
for num_epoch in num_epochs:
    #input_lang, output_lang, train_dataloader = get_dataloader(batch_size)
    train_dataloader = get_dataloader(split="train", input_tokenizer=tokenizer_en, target_tokenizer=tokenizer_de, batch_size=batch_size)
    test_dataloader = get_dataloader(split="test", input_tokenizer=tokenizer_en, target_tokenizer=tokenizer_de, batch_size=batch_size)

    #TODO encoder decoder options
    encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

    train(train_dataloader, encoder, decoder, num_epoch, print_every=5, plot_every=5)
encoder.eval()
decoder.eval()
run_evaluation(encoder, decoder, test_pairs, input_lang, output_lang)

TypeError: 'int' object is not iterable